In [ ]:
import orchid

In [ ]:
# Remember that importing `orchid` results in a call to `pythonnet.load()`
# to specify the targeted runtime:
#
# 'netfx': .NET Framework
# 'coreclr': .NET Core
# 'mono': Mono

import clr

In [ ]:
from System import (
    ArgumentException,
    Array,
    DateTime,
    TimeSpan,
    Int32,
    ValueType,
)

During integration testing, we discovered an issue setting an attribute with
type, `Int32`, using a Python `int` of value 7. The run-time reported that
the types, `Int32` and `PyInt` were incompatible. This scenario requires
significant set up.

In [ ]:
# Find the well named 'Demo_1H'
bakken = orchid.load_project('c:/src/Orchid.IntegrationTestData/frankNstein_Bakken_UTM13_FEET.ifrac')
candidate_wells = list(bakken.wells().find_by_name('Demo_1H'))
assert len(candidate_wells) == 1
demo_1h = candidate_wells[0]

In [ ]:
# Create an attribute with name, 'My New Attribute', and type, `System.Int32`
from Orchid.FractureDiagnostics.Factories.Implementations import Attribute

attribute_to_add_type = Int32
attribute_to_add = Attribute[attribute_to_add_type].Create('My New Attribute')

In [ ]:
# Add newly created attribute to well, 'Demo_1H'
with orchid.dot_net_disposable.disposable(demo_1h.dom_object.ToMutable()) as mutable_well:
    mutable_well.AddStageAttribute(attribute_to_add)

In [ ]:
# Find stage number 7 in well, 'Demo_1H'
maybe_stage = demo_1h.stages().find_by_display_stage_number(7)
assert maybe_stage is not None
stage_7 = maybe_stage

In [ ]:
# Add attribute with value, 17, to stage 7, with Python `int` type.
with (orchid.dot_net_disposable.disposable(stage_7.dom_object.ToMutable())) as mutable_stage:
    # This action will fail because the attribute type is `System.Int32`
    # and `pythonnet-3.0.0.post1` **does not** implicitly equate these two types.
    try:
        mutable_stage.SetAttribute(attribute_to_add, int)
    except ArgumentException as ae:
        print(f'ArgumentException: {ae}')


In [ ]:
# Add attribute with value, 17, to stage 7
with (orchid.dot_net_disposable.disposable(stage_7.dom_object.ToMutable())) as mutable_stage:
    mutable_stage.SetAttribute(attribute_to_add, attribute_to_add_type(7))


In [ ]:
# Verify added attribute value
ignored_object = object()
is_attribute_present, actual_attribute_value = stage_7.dom_object.TryGetAttributeValue(attribute_to_add,
                                                                                       ignored_object)
assert is_attribute_present
assert type(actual_attribute_value) == int
assert actual_attribute_value == 7

### Leakoff.ControlPoints and no ValueType() ctor error

In [ ]:
# noinspection PyUnresolvedReferences
from Orchid.FractureDiagnostics import Leakoff

clr.AddReference('System.Collections')
# noinspection PyUnresolvedReferences
from System.Collections.Generic import List
# noinspection PyUnresolvedReferences
import UnitsNet


In [ ]:
end = DateTime.UtcNow
start = end.Subtract(TimeSpan.FromMinutes(10))
control_points = List[Leakoff.ControlPoint]()

In [ ]:
try:
    control_points.Add(Leakoff.ControlPoint(
        DateTime=start,
        Pressure=UnitsNet.Pressure(95.64, UnitsNet.Units.PressureUnit.PoundForcePerSquareInch)))
    control_points.Add(Leakoff.ControlPoint(
        DateTime=end,
        Pressure=UnitsNet.Pressure(104.8, UnitsNet.Units.PressureUnit.PoundForcePerSquareInch)))
except TypeError as te:
    print(f'TypeError: {te}')


In [ ]:
foo = List[TimeSpan]()
foo.Add(TimeSpan.FromSeconds(314))

In [ ]:
TimeSpan is ValueType

In [ ]:
error_control_points = List[Leakoff.ControlPoint](2)
print(f'{error_control_points.Capacity=}')
print(f'{error_control_points.Count=}')


In [ ]:
try:
    error_control_points.Add(Leakoff.ControlPoint(
        DateTime=start,
        Pressure=UnitsNet.Pressure(95.64, UnitsNet.Units.PressureUnit.PoundForcePerSquareInch)))
    error_control_points.Add(Leakoff.ControlPoint(
        DateTime=end,
        Pressure=UnitsNet.Pressure(104.8, UnitsNet.Units.PressureUnit.PoundForcePerSquareInch)))
except TypeError as te:
    print(f'TypeError: {te}')

In [ ]:
try:
    error_control_points[0] = Leakoff.ControlPoint(
        DateTime=start,
        Pressure=UnitsNet.Pressure(95.64, UnitsNet.Units.PressureUnit.PoundForcePerSquareInch))
except TypeError as te:
    print(f'TypeError: {te}')

In [ ]:
try:
    python_control_points = [
        Leakoff.ControlPoint(
            DateTime=start,
            Pressure=UnitsNet.Pressure(95.64, UnitsNet.Units.PressureUnit.PoundForcePerSquareInch)),
        Leakoff.ControlPoint(
            DateTime=end,
            Pressure=UnitsNet.Pressure(104.8, UnitsNet.Units.PressureUnit.PoundForcePerSquareInch)),
    ]
    error_again_control_points = List[Leakoff.ControlPoint](python_control_points)
except TypeError as te:
    print(f'TypeError: {te}')

In [ ]:
try:
    control_points_array = Array[Leakoff.ControlPoint](2)
    control_points_array[0] = Leakoff.ControlPoint(
            DateTime=start,
            Pressure=UnitsNet.Pressure(95.64, UnitsNet.Units.PressureUnit.PoundForcePerSquareInch))
    control_points_array[1] = Leakoff.ControlPoint(
            DateTime=end,
            Pressure=UnitsNet.Pressure(104.8, UnitsNet.Units.PressureUnit.PoundForcePerSquareInch))
except TypeError as te:
    print(f'TypeError: {te}')

The Orchid team ran some experiments in C#. We observed:

- Code that created a `List<Leakoff.ControlPoint>()` and then using
  `List.Add()` ran without any errors.
- Code that tried to invoke the "constructor",
  `Leakoff.ControlPoint(DateTime, Pressure)` failed to compile.

These observations led us to the hypothesis that `pythonnet-2.5.2` performed
additional work to convert the Python expression
`Leakoff.ControlPoint(DateTime.UtcNow, Pressure.FromPoundsForcePerSquareInt(100))`
into the equivalent C# code:

```
var controlPoint = new Leakoff.ControlPoint();
controlPoint.Date = DateTime.UtcNow;
controlPoint.Pressure = Pressure.FromPoundsForcePerSquareInch(100);
```

The following cell tests that hypothesis.

In [ ]:
python_control_points = [Leakoff.ControlPoint(), Leakoff.ControlPoint()]
python_control_points[0].DateTime = start
python_control_points[0].Pressure = UnitsNet.Pressure(95.64, UnitsNet.Units.PressureUnit.PoundForcePerSquareInch)
python_control_points[1].DateTime = start
python_control_points[1].Pressure = UnitsNet.Pressure(104.8, UnitsNet.Units.PressureUnit.PoundForcePerSquareInch)


In [ ]:
working_control_points = List[Leakoff.ControlPoint]()
working_control_points.Add(python_control_points[0])
working_control_points.Add(python_control_points[1])

In [ ]:
stage_parts = stage_7.dom_object.Parts
type(stage_parts)

In [ ]:
stage_parts.GetType()

In [ ]:
stage_parts.GetType().BaseType.FullName

In [ ]:
net_l = List[Int32]()
net_l.Add(Int32(1))
net_l.GetType().FullName

In [ ]:
net_l_interfaces = net_l.GetType().GetInterfaces()
[i.FullName for i in net_l_interfaces]

[This comment](https://github.com/pythonnet/pythonnet/issues/1153#issuecomment-926143701)
indicates that the `pythonnet-2.5.2` implicit conversions from .NET `List`
to a Python `list` is no longer supported. Consequently, one must use the
explicit but more verbose conversion demonstrated below.

In [ ]:
alt_control_points = List[Leakoff.ControlPoint](len(python_control_points))
for cp in python_control_points:
    alt_control_points.Add(cp)


In [ ]:
for added_cp in alt_control_points:
    print(f'Leakoff.ControlPoint(DateTime={added_cp.DateTime.ToString("o")}, Pressure={added_cp.Pressure}')